<a href="https://colab.research.google.com/github/CaptainCloudd/nonmarkov/blob/main/SVR_Coherence_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.stats import uniform, randint
import ast

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_loc = '/content/drive/MyDrive/Pesquisa_Chile/phase_damping_results.csv'
dataset = pd.read_csv(file_loc)

In [ ]:
# Separing features from labels
dataset['C(t)'] = dataset['C(t)'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
X = np.array(dataset['C(t)'].tolist())
y = dataset[['alpha', 's']].to_numpy()

In [ ]:
# Dividing data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Scaling data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [ ]:
svr = SVR()
Number_param = 5
parametros = {'estimator__kernel': ['rbf'], 'estimator__C': np.linspace(0.001,100, Number_param),
              'estimator__gamma': np.linspace(0.01, 10, Number_param),'estimator__epsilon': np.linspace(0.01, 1,Number_param)}


In [ ]:
multi_output_svr = MultiOutputRegressor(svr)
grid_search = GridSearchCV(estimator=multi_output_svr, param_grid=parametros, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

In [ ]:
grid_search.fit(X_train_scaled, y_train_scaled)
y_pred_scaled = multi_output_svr.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test_scaled)
mse = mean_squared_error(y_test_actual, y_pred, multioutput='raw_values')
print(f"MSE para cada saída: {mse}")
print(f"MSE médio: {mse.mean()}")